# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [2]:
#Your code here
data = pd.read_csv('homepage_actions.csv')
data.head(10)

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view
6,2016-09-24 20:39:03.248853,522116,experiment,view
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
9,2016-09-24 21:00:12.278374,560027,control,view


In [3]:
data['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [4]:
click_ids = set(data[data['action'] == 'click']['id'].unique())
view_ids = set(data[data['action'] == 'view']['id'].unique())

print("Unique number of id's: {} clickers, {} viewers".format(len(click_ids), len(view_ids)))
print("Number of viewers who didn't click: {}".format(len(view_ids-click_ids)))
print("Number of clickers who didn't view: {}".format(len(click_ids-view_ids)))
# everyone who clicked also viewed the homepage

Unique number of id's: 1860 clickers, 6328 viewers
Number of viewers who didn't click: 4468
Number of clickers who didn't view: 0


In [5]:
control_ids = set(data[data['group'] == 'control']['id'].unique())
experiment_ids = set(data[data['group'] == 'experiment']['id'].unique())

print("Unique number of id's: {} in control group, {} in experiment group".format(len(control_ids), 
                                                                                  len(experiment_ids)))
print("Number of id's that overlap: {} id's".format(len(control_ids & experiment_ids)))
# no overlap between control and experiment groups

Unique number of id's: 3332 in control group, 2996 in experiment group
Number of id's that overlap: 0 id's


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [6]:
#Your code here

# H0 : control and experiment groups have same click rates
# Ha : experiment groups have more click rates

data['count'] = 1
control = data[data['group'] == 'control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = data[data['group'] == 'experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)

print('Control:')
print('Number of clicks: ', sum(control['click']))
print('Number of views: ', sum(control['view']))
print('Click rate: ', control['click'].mean())

print('\n')
print('Experiment:')
print('Number of clicks: ', sum(experiment['click']))
print('Number of views: ', sum(experiment['view']))
print('Click rate: ', experiment['click'].mean())

Control:
Number of clicks:  932.0
Number of views:  3332.0
Click rate:  0.2797118847539016


Experiment:
Number of clicks:  928.0
Number of views:  2996.0
Click rate:  0.3097463284379172


In [7]:
def welch_t(a, b):
    
    """ Calculate Welch's t statistic for two samples. """
    
    # “ddof = Delta Degrees of Freedom”: the divisor used in the calculation is N - ddof, 
    #  where N represents the number of elements. By default ddof is zero.
    
    x1_bar = a.mean()
    var1 = a.var(ddof=1) 
    n1 = len(a)
    
    x2_bar = b.mean()
    var2 = b.var(ddof=1) 
    n2 = len(b)
    
    t = np.abs(x1_bar - x2_bar) / np.sqrt(var1/n1 + var2/n2) 
    
    return t

def welch_df(a, b):
    
    """ Calculate the effective degrees of freedom for two samples. """
    var1 = a.var(ddof=1) 
    n1 = len(a)
    
    var2 = b.var(ddof=1) 
    n2 = len(b)
    
    df = (var1/n1 + var2/n2)**2 / ((var1**2/((n1-1)*n1**2)) + (var2**2/((n2-1)*n2**2)))
    return df #Return the degrees of freedom

In [8]:
t = welch_t(control['click'], experiment['click'])
df = welch_df(control['click'], experiment['click'])

p = 1 - stats.t.cdf(t, df)
print(p)

0.004466402814337078


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [9]:
#Your code here
control_clickrate = control['click'].mean()
experiment_expected_clicks = len(experiment) * control_clickrate
experiment_expected_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [10]:
#Your code here
n = len(experiment)
p = control_clickrate

experiment_std = np.sqrt(n*p*(1-p))
experiment_std

24.568547907005815

In [11]:
experiment_actual_clicks = experiment['click'].sum()
z = (experiment_actual_clicks - experiment_expected_clicks)/experiment_std
z

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [12]:
#Your code here
results = stats.norm.sf(z)
results

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: The p-value using the normal distribution based on the z-score is lower. However, both methods yield statistically significant results, rejecting the null hypothesis. The experimental group seems to be a more effective homepage.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.